<a href="https://colab.research.google.com/github/fpaterni10/projeto-aplicado-iv-desemprego-br/blob/main/projeto/cd_projeto_aplicado_IV_entrega_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

  <img src="https://raw.githubusercontent.com/scalabrinig/cdProjetoAplicadoIV/d093146488f56dfcf0ef286bcee8efe0e71b9c76/figuras/mackenzie_logo.jpg" width="25%" align="right"/>

# **PROJETO APLICADO IV - Ciência de Dados EaD - 2025/02**


# **Entrega 2**

---

# **Previsão da Taxa de Desocupação com Modelos de Séries Temporais**:


---

In [ ]:
#@title **Identificação do Grupo e Opção do Projeto**

#@markdown Integrantes do Grupo, nome completo em ordem alfabética (*informe: \<nome\>, \<matrícula\>*)
Aluno1 = 'Aline Correa de Araújo, 10414773' #@param {type:"string"}
Aluno2 = 'Franciele Paterni, 10414598' #@param {type:"string"}
Aluno3 = 'Giovanna Sobral da Silva, 10424600' #@param {type:"string"}
Aluno4 = 'Guilherme Soares Frota, 10416060' #@param {type:"string"}

# **Introdução**


A taxa de desocupação é um dos principais indicadores socioeconômicos utilizados para medir a saúde do mercado de trabalho. No Brasil, esse índice reflete diretamente o poder de compra, a renda das famílias e a qualidade de vida da população. Os movimentos da desocupação estão fortemente associados a fatores macroeconômicos, como política monetária, taxa básica de juros (SELIC), inflação e geração de empregos formais.

Ao longo da última década, a taxa de desemprego tem sofrido oscilações expressivas, principalmente em períodos de crise econômica e de retração do mercado formal. Esses ciclos impactam o consumo interno, aumentam a informalidade e acentuam desigualdades sociais (CASAGRANDE et al., 2016). Compreender o comportamento da taxa de desocupação e prever sua evolução é estratégico para subsidiar a formulação de políticas públicas, orientar investimentos e apoiar ações de requalificação profissional.

Nesse sentido, a previsão de séries temporais tem se mostrado uma abordagem promissora. Modelos como ARIMA, SARIMA e SARIMAX vêm sendo amplamente utilizados para problemas de previsão em economia e mercado de trabalho (OLIVEIRA; ALBARRACIN; SILVA, 2023). No caso brasileiro, os microdados do Cadastro Geral de Empregados e Desempregados (CAGED) representam uma base robusta, com registros mensais de admissões e desligamentos desde 2012, permitindo análises detalhadas sobre o saldo de empregos formais. A taxa SELIC, por sua vez, é um determinante macroeconômico fundamental, influenciando crédito, consumo e geração de postos de trabalho.

O presente projeto tem como objetivo geral prever a taxa de desocupação mensal no Brasil para o ano de 2025, utilizando modelos de séries temporais baseados nos microdados do CAGED (2012–2025) e incorporando variáveis exógenas como a taxa SELIC, obtida junto ao Banco Central do Brasil. Como objetivos específicos, destacam-se:

Consolidar e tratar uma base mensal com os microdados do CAGED (2012–2025).

Incorporar variáveis exógenas relevantes (como a SELIC) no modelo de previsão.

Analisar as propriedades estatísticas da série (estacionariedade, sazonalidade, linearidade).

Testar modelos ARIMA, SARIMA e SARIMAX em diferentes configurações.

Comparar o desempenho dos modelos com e sem variáveis exógenas.

Validar a capacidade preditiva com dados históricos e projetar valores para 2025.

A justificativa para a escolha desta metodologia baseia-se na necessidade de integrar granularidade dos microdados trabalhistas com fatores macroeconômicos. Essa combinação permite análises mais robustas e aplicáveis, oferecendo resultados que podem ser utilizados por gestores públicos, pesquisadores e pela sociedade em geral para antecipar tendências do mercado de trabalho brasileiro.

# **Referencial Teórico**

A previsão da taxa de desocupação com base em séries temporais constitui uma abordagem consolidada para análise da dinâmica do mercado de trabalho. Modelos clássicos de séries temporais — ARIMA, SARIMA e suas extensões com regressores externos (SARIMAX) — modelam explicitamente componentes de tendência, sazonalidade e ruído, sendo amplamente utilizados em problemas macroeconômicos e laborais (HYNDMAN; ATHANASOPOULOS, 2018). Quando a série apresenta sazonalidade periódica, a extensão SARIMA é indicada; para incorporar variáveis exógenas (por exemplo, taxa SELIC, PIB, indicadores setoriais) utiliza-se SARIMAX, que permite avaliar o efeito de covariáveis sobre a dinâmica temporal da variável-alvo.

Além dos modelos lineares de séries temporais, metodologias baseadas em redes neurais recorrentes (RNN, LSTM) têm sido aplicadas em cenários onde não linearidades complexas ou interações multivariadas exigem maior capacidade de representação. Entretanto, redes neurais demandam volume de dados e cuidado extra no ajuste de hiperparâmetros e validação temporal (VALENTINI et al., 2024). Para séries econômicas com estrutura sazonal e poucos regressores observáveis, modelos SARIMA/SARIMAX tendem a apresentar boa interpretabilidade e robustez (HYNDMAN; ATHANASOPOULOS, 2018).

No contexto brasileiro, os microdados do CAGED, disponibilizados pelo Ministério do Trabalho e Emprego, fornecem informações detalhadas sobre admissões, desligamentos e saldo do emprego formal. Esses dados permitem derivar indicadores que se correlacionam diretamente com a taxa de desocupação nacional. Já a taxa SELIC, divulgada pelo Banco Central do Brasil, é considerada um dos principais determinantes macroeconômicos da atividade econômica, afetando diretamente o crédito, os investimentos e a geração de postos de trabalho.

Estudos nacionais demonstram aplicações bem-sucedidas de modelos de séries temporais na previsão da taxa de desemprego. Pesquisas que compararam ARIMA/SARIMA com abordagens híbridas (ex.: modelos lineares combinados com redes neurais) indicam que modelos lineares bem especificados frequentemente entregam previsões competitivas, enquanto arquiteturas mais complexas podem superar em contextos com grandes volumes de dados multivariados e padrões não lineares persistentes (CASAGRANDE et al., 2016; VALENTINI et al., 2024).

Síntese das escolhas metodológicas para este projeto

Usaremos SARIMA como modelo de referência quando a série agregada apresentar sazonalidade clara; adotaremos SARIMAX para incorporar variáveis exógenas (SELIC e saldo do CAGED). Essas escolhas privilegiam interpretação e robustez em amostras históricas de extensão moderada (HYNDMAN; ATHANASOPOULOS, 2018).

Consideraremos modelos recorrentes (LSTM) como alternativa complementar para avaliar possíveis ganhos de performance em relação aos modelos lineares.

A etapa de pré-processamento enfatizará controle de qualidade dos microdados (consolidação CAGED 2012–2025), tratamento de valores ausentes/outliers, diferenciação para estacionariedade e análise de autocorrelações para identificação das ordens (p, d, q) e (P, D, Q, s).


# **Pipeline da Solução**


1. **Aquisição dos dados**  
   - Coleta dos microdados do **CAGED (2012–2025)** e da **taxa SELIC**.  
   - Armazenamento inicial em `data/raw/`.

2. **Pré-processamento**  
   - Padronização de datas.  
   - Tratamento de valores ausentes e outliers.  
   - Consolidação em série temporal mensal em `data/processed/`.

3. **Análise exploratória e decomposição**  
   - Visualização de tendências, sazonalidade e resíduos.  
   - Gráficos de linha, ACF e PACF.

4. **Engenharia de variáveis exógenas**  
   - Inclusão da SELIC.  
   - Criação de defasagens (lags) e indicadores sazonais.

5. **Modelagem e validação temporal**  
   - Testes com **Naive**, **ARIMA**, **SARIMA** e **SARIMAX**.  
   - Validação com técnica rolling/walk-forward.

6. **Avaliação e geração de previsões**  
   - Métricas: MAE, RMSE, MAPE.  
   - Forecast para 2025 com intervalos de confiança.  
   - Armazenamento em `results/`.

7. **Documentação e entrega**  
   - Organização dos notebooks.  
   - Consolidação de figuras, tabelas e instruções no `README.md`.



# Cronograma

##  Etapas 1 e 2:  (20/08/2025 — 26/09/2025)
| Nº | Atividade | Responsável(s) | Data início | Data término | Status |
|----|-----------|----------------|-------------|--------------|--------|
| 1 | Definição do título e escopo do projeto | Todos | 20/08/2025 | 07/09/2025 | Concluído |
| 2 | Identificação do grupo (nomes e matrículas) | Todos | 20/08/2025 | 07/09/2025 | Concluído |
| 3 | Redação da Introdução, Motivação e Justificativa | Todos | 20/08/2025 | 07/09/2025 | Concluído |
| 4 | Formulação de Objetivo Geral e Objetivos Específicos | Todos | 20/08/2025 | 07/09/2025 | Concluído |
| 5 | Descrição da base de dados e variáveis exógenas (CAGED, SELIC) | Franciele / Guilherme | 20/08/2025 | 07/09/2025 | Concluído |
| 6 | Justificativa metodológica e bibliografia inicial | Aline / Giovanna | 20/08/2025 | 07/09/2025 | Concluído |
| 7 | Planejamento inicial (pipeline + subetapas) | Todos | 20/08/2025 | 25/09/2025 | Concluído |
| 8 | Elaboração da Entrega 2 (introdução, referencial, pipeline, cronograma) | Todos | 10/09/2025 | 25/09/2025 | Concluído |

---

## Etapa 3: Implementação Parcial (25/09/2025 — 31/10/2025)
| Nº | Atividade | Responsável(s) | Data início | Data término | Status |
|----|-----------|----------------|-------------|--------------|--------|
| 9 | Aquisição e integração dos arquivos brutos (CAGED + SELIC) | Guilherme / Fran | 25/09/2025 | 28/09/2025 | Em andamento |
| 10 | Pré-processamento e montagem da série mensal consolidada | Fran / Aline | 29/09/2025 | 06/10/2025 | Planejado |
| 11 | Engenharia de features exógenas (SELIC, lags, médias móveis) | Guilherme / Giovanna | 29/09/2025 | 06/10/2025 | Planejado |
| 12 | Análise Exploratória de Dados (EDA) e visualizações | Giovanna / Aline | 07/10/2025 | 13/10/2025 | Planejado |
| 13 | Diagrama da solução (fluxo visual do pipeline) | Todos | 07/10/2025 | 10/10/2025 | Planejado |
| 14 | Definição final dos modelos candidatos (ARIMA, SARIMA, SARIMAX) | Todos | 14/10/2025 | 16/10/2025 | Planejado |
| 15 | Treinamento inicial dos modelos clássicos | Aline / Guilherme | 17/10/2025 | 20/10/2025 | Planejado |
| 16 | Treinamento de baseline com redes neurais (opcional) | Giovanna / Fran | 21/10/2025 | 24/10/2025 | Planejado |
| 17 | Avaliação preliminar e ajustes iniciais | Todos | 25/10/2025 | 28/10/2025 | Planejado |
| 18 | Consolidação da Entrega 3 (notebook parcial) | Todos | 29/10/2025 | 31/10/2025 | Planejado |

---

## Etapa 4: Implementação Final (01/11/2025 — 28/11/2025)
| Nº | Atividade | Responsável(s) | Data início | Data término | Status |
|----|-----------|----------------|-------------|--------------|--------|
| 19 | Avaliação final e tuning dos modelos | Todos | 01/11/2025 | 05/11/2025 | Planejado |
| 20 | Geração dos resultados finais (métricas + gráficos + forecast 2025) | Fran / Aline | 06/11/2025 | 10/11/2025 | Planejado |
| 21 | Discussão e conclusão crítica (qualidades, limitações, melhorias) | Giovanna / Guilherme | 11/11/2025 | 15/11/2025 | Planejado |
| 22 | Redação dos resultados e organização no notebook | Todos | 16/11/2025 | 20/11/2025 | Planejado |
| 23 | Revisão final e formatação ABNT nas referências | Todos | 21/11/2025 | 23/11/2025 | Planejado |
| 24 | Preparação da apresentação (slides + vídeo) | Todos | 24/11/2025 | 26/11/2025 | Planejado |
| 25 | Submissão final no GitHub + vídeo | Todos | 27/11/2025 | 28/11/2025 | Planejado |



**Referências**

- HYNDMAN, R. J.; ATHANASOPOULOS, G. *Forecasting: principles and practice.* 2ª ed. Online.  
- OLIVEIRA, R.; ALBARRACIN, O. Y.; SILVA, G. R. *Introdução às Séries Temporais: Uma Abordagem Prática em Python.* 2023.  

- CASAGRANDE, D. L.; OLIVEIRA, F. R.; STUDART, G.; et al. *Métodos de previsão para a taxa de desemprego mensal: uma análise de séries temporais.* Revista de Economia, 2016.  

- BRASIL. Ministério do Trabalho e Emprego. **Cadastro Geral de Empregados e Desempregados – CAGED**. Brasília: MTE, 2012–2025. Disponível em: <http://pdet.mte.gov.br/>. Acesso em: set. 2025.  

- BANCO CENTRAL DO BRASIL. *Taxa SELIC – histórico mensal.* Brasília, 2012–2025.  



In [ ]:
#@title **Avaliação**
Referencial_teorico = 10 #@param {type:"slider", min:0, max:10, step:1}

Pipeline_solucao = 10 #@param {type:"slider", min:0, max:10, step:1}

Cronograma = 10 #@param {type:"slider", min:0, max:10, step:1}

In [ ]:
#@title **Nota Final**
nota = 0.45*Referencial_teorico + 0.45*Pipeline_solucao + 0.10*Cronograma

print(f'Nota final do trabalho {nota :.1f}')

import numpy as np
import pandas as pd

alunos = pd.DataFrame()

lista_nome = []

for i in range(1,6):
  exec("if Aluno" + str(i) + " !='None':  lista = Aluno" + str(i) + ".split(','); lista_nome.append(lista[0]);")

alunos['nome'] = lista_nome
alunos['nota'] = np.round(nota,1)
print()
display(alunos)

Nota final do trabalho 10.0



,nome,nota
0,Aluno 1,10.0
1,Aluno 2,10.0
2,Aluno 3,10.0
3,Aluno 4,10.0
4,Aluno 5,10.0
